In [ ]:
#Current issues/todo:
#add element of time (Obama vs Trump or other)
#calculating jaccard, coherence, perplexity
#attempt BTM
#compare all three (LDA,MNF,BTM)

In [1]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
#warnings.filterwarnings('ignore')

#warning website: https://queirozf.com/entries/suppressing-ignoring-warnings-in-python-reference-and-examples

In [2]:
import pandas as pd
import numpy as np
import re
import string

from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

import pyLDAvis
import pyLDAvis.gensim as gensimvis
import pyLDAvis.sklearn

In [17]:
# Way around missing packages that for some reason I can't download the normal way

import os
import nltk

# Create NLTK data directory
NLTK_DATA_DIR = './nltk_data'
if not os.path.exists(NLTK_DATA_DIR):
        os.makedirs(NLTK_DATA_DIR)

nltk.data.path.append(NLTK_DATA_DIR)

# Download packages and store in directory above
nltk.download('punkt', download_dir=NLTK_DATA_DIR)
nltk.download('averaged_perceptron_tagger', download_dir=NLTK_DATA_DIR)
nltk.download('wordnet', download_dir=NLTK_DATA_DIR)

[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ./nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to ./nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
df = pd.read_csv("data/doj.csv")
df = df.drop(columns=['id','topics','components'], axis=1)

In [19]:
df['contents'] = df['contents'].astype(str)
df['contents_processed'] = \
df['contents'].map(lambda x: re.sub('[,\.!?]','',x))
df['contents_processed'] = \
df['contents_processed'].map(lambda x: x.lower())
df['contents_processed']

df['title'] = df['title'].astype(str)
df['title_processed'] = \
df['title'].map(lambda x: re.sub('[,\.!?]','',x))
df['title_processed'] = \
df['title_processed'].map(lambda x: x.lower())
df['title_processed']

df = df.drop(columns=['title', 'contents'])
df

<>:3: DeprecationWarning: invalid escape sequence \.
<>:10: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<>:10: DeprecationWarning: invalid escape sequence \.
/var/folders/fm/xwcj8f4d2xl395kr3dnsf8z40000gn/T/ipykernel_24647/202145478.py:3: DeprecationWarning: invalid escape sequence \.
  df['contents'].map(lambda x: re.sub('[,\.!?]','',x))
/var/folders/fm/xwcj8f4d2xl395kr3dnsf8z40000gn/T/ipykernel_24647/202145478.py:10: DeprecationWarning: invalid escape sequence \.
  df['title'].map(lambda x: re.sub('[,\.!?]','',x))


,date,contents_processed,title_processed
0,2018-07-27,the us district court for the northern distric...,district court enters permanent injunction aga...
1,2018-07-27,the department of justice announced today that...,justice department announces resolution with ...
2,2018-07-27,yesterday a federal judge found ahmed el-sheri...,kansas city area laboratory owner convicted of...
3,2018-07-26,the department of justice announced today that...,3m company agrees to pay $91 million to resolv...
4,2018-07-26,a federal grand jury in new orleans louisiana ...,amite residents charged with civil rights crim...
...,...,...,...
17819,2018-07-30,washington – a jordanian national residing in ...,jordanian national arrested in new york to fac...
17820,2018-07-30,a charlottesville virginia man was sentenced t...,former virginia high school science teacher se...
17821,2018-07-30,a federal jury convicted a sterling virginia w...,former owner of sleep study businesses convict...
17822,2018-07-30,remarks as prepared for delivery\r\nthank you ...,attorney general sessions delivers remarks at ...


In [29]:
#setting max rows to see more throughout process

pd.set_option('display.max_rows', 50)

#looking at vectorizor in order to see what words might need to be added to the stop word list
text = " ".join(review for review in df.contents_processed)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english')
matrix = vectorizer.fit_transform([text])

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
sorted_count = counts.T.sort_values(by=0, ascending=False).head(20)
sorted_count['word'] = sorted_count.index
sorted_list = list(sorted_count['word'])
sorted_list

['department',
 'attorney',
 'district',
 'justice',
 'division',
 'general',
 'criminal',
 'office',
 'assistant',
 'federal',
 'fraud',
 'tax',
 'today',
 'enforcement',
 'case',
 'states',
 'court',
 'law',
 'united',
 'said']

In [21]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

/opt/homebrew/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
/opt/homebrew/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachaellam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
stop_words = stopwords.words('english')
stop_words.extend([sorted_list,'us', 'years', 'one', 'also'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
            if word not in stop_words] for doc in texts]

doj_data = df.contents_processed.values.tolist()
doj_words = list(sent_to_words(doj_data))
# remove stop words
doj_words = remove_stopwords(doj_words)
print(doj_words[:1][0][:30])

['district', 'court', 'northern', 'district', 'illinois', 'entered', 'permanent', 'injunction', 'three', 'related', 'chicago', 'companies', 'global', 'marketing', 'enterprises', 'inc', 'lifeline', 'nutrients', 'corp', 'pronto', 'foods', 'company', 'well', 'owner', 'eduardo', 'chua', 'operations', 'manager', 'haidee', 'dawis']


In [31]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(doj_words)
# Create Corpus
texts = doj_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 2), (23, 2), (24, 6), (25, 1), (26, 3), (27, 2), (28, 1), (29, 1)]


In [32]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"department" + 0.009*"attorney" + 0.008*"division" + 0.006*"district" '
  '+ 0.006*"justice" + 0.005*"criminal" + 0.005*"law" + 0.005*"federal" + '
  '0.005*"fraud" + 0.005*"assistant"'),
 (1,
  '0.009*"department" + 0.009*"attorney" + 0.008*"justice" + 0.005*"assistant" '
  '+ 0.005*"general" + 0.005*"division" + 0.005*"criminal" + 0.005*"today" + '
  '0.005*"enforcement" + 0.005*"federal"'),
 (2,
  '0.009*"attorney" + 0.009*"department" + 0.008*"division" + 0.008*"district" '
  '+ 0.007*"justice" + 0.006*"federal" + 0.006*"enforcement" + 0.005*"today" + '
  '0.005*"law" + 0.004*"assistant"'),
 (3,
  '0.011*"department" + 0.010*"attorney" + 0.009*"justice" + 0.008*"district" '
  '+ 0.006*"division" + 0.005*"federal" + 0.005*"tax" + 0.005*"office" + '
  '0.005*"assistant" + 0.004*"law"'),
 (4,
  '0.012*"attorney" + 0.009*"department" + 0.009*"division" + 0.008*"district" '
  '+ 0.007*"general" + 0.006*"office" + 0.006*"fraud" + 0.006*"justice" + '
  '0.006*"assistant" + 0

In [33]:
#LDA coherence

cm = gensim.models.coherencemodel.CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
print(cm.get_coherence())

0.5495600845982085


In [34]:
#LDA perplexity

print(lda_model.log_perplexity(corpus))

-8.31602578764688


In [35]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(LDAvis_prepared)

/opt/homebrew/lib/python3.9/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/opt/homebrew/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the im

In [36]:
# tokenizing nouns for LDA
def nouns(text):
    is_noun = lambda pos : pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
                for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

data_nouns = pd.DataFrame(df.contents_processed.apply(nouns))

data_nouns.head()

,contents_processed
0,district court district illinois injunction ch...
1,department justice today mirelis sa resolution...
2,yesterday judge owner operator radiation safet...
3,department justice today company st paul minne...
4,jury orleans trafficking hate crime indictment...


In [61]:
doj_nouns_data = data_nouns.contents_processed.values.tolist()
doj_nouns_words = list(sent_to_words(doj_nouns_data))
# remove stop words
doj_noun_words = remove_stopwords(doj_nouns_words)
print(doj_nouns_words[:1][0][:30])

['district', 'court', 'district', 'illinois', 'injunction', 'chicago', 'company', 'marketing', 'enterprise', 'nutrient', 'corp', 'pronto', 'food', 'company', 'owner', 'chua', 'operation', 'manager', 'haidee', 'injunction', 'defendant', 'selling', 'distributing', 'supplement', 'drug', 'department', 'justice', 'today', 'department', 'complaint']


In [62]:
# Create Dictionary with doj_nouns
id2word_nouns = corpora.Dictionary(doj_nouns_words)
# Create Corpus
texts_nouns = doj_nouns_words
# Term Document Frequency
corpus_nouns = [id2word_nouns.doc2bow(text) for text in texts_nouns]
# View
print(corpus_nouns[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 2), (9, 6), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 7), (18, 1), (19, 1), (20, 3), (21, 6), (22, 3), (23, 1), (24, 4), (25, 1), (26, 7), (27, 1), (28, 2), (29, 1)]


In [64]:
# Build LDA model with nouns
num_topics = 15
nouns_lda_model = gensim.models.LdaMulticore(corpus=corpus_nouns,
                                       id2word=id2word_nouns,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(nouns_lda_model.print_topics())
nouns_doc_lda = nouns_lda_model[corpus]

[(0,
  '0.021*"district" + 0.016*"attorney" + 0.015*"department" + 0.011*"division" '
  '+ 0.011*"year" + 0.010*"sentence" + 0.009*"right" + 0.009*"justice" + '
  '0.008*"office" + 0.008*"today"'),
 (1,
  '0.015*"department" + 0.010*"attorney" + 0.009*"office" + 0.009*"law" + '
  '0.008*"case" + 0.008*"today" + 0.008*"information" + 0.007*"justice" + '
  '0.007*"division" + 0.007*"district"'),
 (2,
  '0.015*"department" + 0.015*"attorney" + 0.012*"district" + 0.010*"office" + '
  '0.010*"drug" + 0.010*"case" + 0.009*"charge" + 0.009*"division" + '
  '0.009*"company" + 0.008*"investigation"'),
 (3,
  '0.012*"case" + 0.011*"state" + 0.010*"attorney" + 0.010*"department" + '
  '0.009*"district" + 0.009*"law" + 0.008*"service" + 0.008*"year" + '
  '0.007*"defendant" + 0.007*"justice"'),
 (4,
  '0.019*"department" + 0.017*"attorney" + 0.014*"division" + 0.014*"health" + '
  '0.013*"district" + 0.012*"justice" + 0.012*"fraud" + 0.011*"service" + '
  '0.010*"office" + 0.009*"state"'),
 (5,
  

In [101]:
LDAvis_prepared_nouns = pyLDAvis.gensim.prepare(nouns_lda_model, corpus_nouns, id2word_nouns)

In [102]:
pyLDAvis.display(LDAvis_prepared_nouns)

In [106]:
#removing extra stop words
stop_noun = ['america', 'today', 'thing', 'wwwjusticegov']
stop_words_noun_agg = text.ENGLISH_STOP_WORDS.union(stop_noun)

tv_noun = TfidfVectorizer(stop_words=stop_words_noun_agg, ngram_range = (1,1), max_df = 0.9, min_df = 0.1)

data_tv_noun = tv_noun.fit_transform(data_nouns.contents_processed)

data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns = tv_noun.get_feature_names())

data_dtm_noun.index = df.index

data_dtm_noun.head()

,abuse,access,account,act,action,activity,addition,administration,agency,agent,...,use,victim,violation,visit,washington,website,wire,work,year,york
0,0.000000,0.0,0.000000,0.046878,0.111030,0.000000,0.000000,0.0,0.00000,0.000000,...,0.06788,0.0,0.059631,0.073862,0.000000,0.061870,0.0,0.000000,0.000000,0.0
1,0.000000,0.0,0.875006,0.000000,0.000000,0.037355,0.020835,0.0,0.00000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.011992,0.0,0.000000,0.006879,0.0
2,0.000000,0.0,0.000000,0.217439,0.085834,0.000000,0.000000,0.0,0.08190,0.131620,...,0.00000,0.0,0.184396,0.000000,0.082576,0.000000,0.0,0.096909,0.109742,0.0
3,0.186650,0.0,0.000000,0.113259,0.067064,0.000000,0.000000,0.0,0.06399,0.102837,...,0.08200,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.102186,0.0,0.000000,0.124013,0.000000,0.000000,0.071088,0.0,0.00000,0.000000,...,0.00000,0.0,0.552129,0.000000,0.000000,0.000000,0.0,0.000000,0.140827,0.0


In [110]:
#iterate through topics
def display_topics(model, feature_names, num_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic", ix)
        else:
            print("\nTopic: '", topic_names[ix], "'")
        print(",".join([feature_names[i] \
            for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [111]:
nmf_model = NMF(10)

doc_topic = nmf_model.fit_transform(data_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)


Topic 0
district,prison,case,division,trial,office,court,judge,charge,plea

Topic 1
tax,return,irs,income,business,revenue,customer,division,deputy,year

Topic 2
health,care,fraud,claim,service,program,home,provider,office,force

Topic 3
settlement,company,state,agreement,act,claim,government,lawsuit,complaint,compliance

Topic 4
right,officer,division,county,complaint,act,law,lawsuit,person,district

Topic 5
law,enforcement,community,crime,state,officer,agency,task,program,force

Topic 6
indictment,defendant,count,charge,conspiracy,jury,year,prison,fine,fbi

Topic 7
fraud,company,bank,scheme,account,money,conspiracy,wire,payment,business

Topic 8
victim,crime,defendant,abuse,family,home,case,initiative,individual,prosecution

Topic 9
drug,complaint,defendant,administration,member,organization,enforcement,office,operation,district


In [118]:
# running into issues with the modelling

LDAvis_prepared_nouns = pyLDAvis.sklearn.prepare(nmf_model, data_tv_noun, tv_noun)
pyLDAvis.display(LDAvis_prepared_nouns)

ValidationError: 
 * Not all rows (distributions) in doc_topic_dists sum to 1.

In [129]:
#splitting documents by presidency
df.dropna(subset = ['title', 'date'], inplace=True)
obama = df.loc[df['date'].str.contains('2013|2014|2015|2016')]
trump = df.loc[df['date'].str5.contains('2017|2018|2019|2020')]

In [137]:
# obama vectorization

obama_nouns = pd.DataFrame(obama.contents_processed.apply(nouns))

obama_tv_noun = tv_noun.fit_transform(obama_nouns.contents_processed)

obama_dtm_noun = pd.DataFrame(obama_tv_noun.toarray(), columns = tv_noun.get_feature_names())

obama_dtm_noun.index = obama.index

obama_dtm_noun.head()

,access,account,act,action,activity,addition,administration,agency,agent,agreement,...,use,victim,violation,visit,washington,website,wire,work,year,york
2208,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053586,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.075679,0.000000,0.000000,0.093784,0.0
2209,0.0,0.000000,0.035346,0.081531,0.000000,0.038378,0.000000,0.000000,0.185083,0.154334,...,0.000000,0.000000,0.04427,0.000000,0.053239,0.000000,0.056017,0.000000,0.000000,0.0
2210,0.0,0.248852,0.036102,0.000000,0.073647,0.019599,0.026409,0.000000,0.000000,0.059113,...,0.025294,0.000000,0.00000,0.000000,0.000000,0.022248,0.000000,0.023981,0.000000,0.0
2211,0.0,0.000000,0.238984,0.000000,0.054169,0.000000,0.000000,0.044411,0.000000,0.000000,...,0.000000,0.048722,0.00000,0.059589,0.000000,0.049092,0.000000,0.000000,0.000000,0.0
2212,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065646,0.082110,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.092712,0.119210,0.000000,0.057446,0.0


In [133]:
nmf_model = NMF(10)

obama_doc_topic = nmf_model.fit_transform(obama_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)


Topic 0
district,indictment,charge,conspiracy,defendant,count,division,case,office,fraud

Topic 1
tax,return,income,irs,business,revenue,account,year,bank,customer

Topic 2
health,care,fraud,claim,service,home,program,office,provider,center

Topic 3
settlement,company,state,agreement,act,government,claim,complaint,lawsuit,program

Topic 4
right,officer,law,police,community,county,victim,enforcement,division,district


In [134]:
# trump vectorization

trump_nouns = pd.DataFrame(trump.contents_processed.apply(nouns))

trump_tv_noun = tv_noun.fit_transform(trump_nouns.contents_processed)

trump_dtm_noun = pd.DataFrame(trump_tv_noun.toarray(), columns = tv_noun.get_feature_names())

trump_dtm_noun.index = trump.index

trump_dtm_noun.head()

,abuse,access,account,act,action,activity,addition,administration,affair,agency,...,visit,washington,way,website,william,wire,woman,work,year,york
0,0.000000,0.0,0.000000,0.047643,0.112072,0.000000,0.000000,0.0,0.074045,0.000000,...,0.074901,0.000000,0.0,0.056004,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0.000000,0.0,0.869922,0.000000,0.000000,0.037663,0.021680,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.011087,0.0,0.0,0.000000,0.000000,0.007033,0.0
2,0.000000,0.0,0.000000,0.221367,0.086787,0.000000,0.000000,0.0,0.000000,0.077542,...,0.000000,0.106764,0.0,0.000000,0.0,0.0,0.000000,0.089854,0.110043,0.0
3,0.165542,0.0,0.000000,0.113144,0.066537,0.000000,0.000000,0.0,0.000000,0.059449,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.178048,0.000000,0.000000,0.0
4,0.090695,0.0,0.000000,0.123976,0.000000,0.000000,0.071243,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.097547,0.000000,0.138666,0.0


In [136]:
nmf_model = NMF(10)

trump_doc_topic = nmf_model.fit_transform(trump_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)


Topic 0
company,district,security,division,office,charge,plea,conspiracy,case,fraud

Topic 1
tax,return,irs,income,deputy,division,business,assistant,revenue,prison

Topic 2
fraud,health,care,service,force,claim,scheme,program,home,inspector

Topic 3
settlement,claim,company,state,act,agreement,government,allegation,program,service

Topic 4
law,enforcement,crime,state,community,officer,police,people,agency,president

Topic 5
victim,crime,defendant,violence,fraud,woman,service,scheme,prosecution,office

Topic 6
right,officer,division,district,complaint,court,law,force,violation,immigration

Topic 7
child,district,abuse,case,visit,initiative,police,county,resource,office

Topic 8
indictment,defendant,count,charge,conspiracy,jury,doubt,police,allegation,member

Topic 9
drug,defendant,complaint,district,administration,office,enforcement,operation,organization,member
